In [ ]:
# Customizable parameters - modify these to test different scenarios
NUM_EXAMPLES = 200
PROPOSED_FUSION_RATE = 0.65  # What fraction of proposed decisions are fusion
BASELINE_ERROR_RATE = 0.08   # Baseline error rate
PROPOSED_ERROR_RATE = 0.09   # Proposed method error rate

def generate_custom_data(n_examples, fusion_rate, baseline_error_rate, proposed_error_rate):
    """Generate custom evaluation data with specified parameters."""
    
    # Baseline: all fission decisions
    baseline_data = []
    for i in range(n_examples):
        error_flag = i < int(n_examples * baseline_error_rate)
        baseline_data.append({"decision": "fission", "error": error_flag})
    
    # Proposed: mix of fusion/fission
    proposed_data = []
    n_fusion = int(n_examples * fusion_rate)
    for i in range(n_examples):
        decision = "fusion" if i < n_fusion else "fission"
        error_flag = i < int(n_examples * proposed_error_rate)
        proposed_data.append({"decision": decision, "error": error_flag})
    
    return {"baseline": baseline_data, "proposed": proposed_data}

# Generate new data with custom parameters
custom_results = generate_custom_data(NUM_EXAMPLES, PROPOSED_FUSION_RATE, BASELINE_ERROR_RATE, PROPOSED_ERROR_RATE)
custom_metrics = compute_metrics(custom_results)

print(f"🔧 Custom Analysis Results (Fusion Rate: {PROPOSED_FUSION_RATE:.0%}):")
print(f"   API Reduction: {custom_metrics['improvement']['api_reduction_pct']:.1f}%")
print(f"   Error Rate Change: {custom_metrics['improvement']['error_rate_diff']:.2%}")

# Quick comparison function
def compare_fusion_rates(rates):
    """Compare API reduction for different fusion rates."""
    print("\n🔍 Fusion Rate Sensitivity Analysis:")
    print("-" * 40)
    for rate in rates:
        test_results = generate_custom_data(200, rate, 0.08, 0.09)
        test_metrics = compute_metrics(test_results)
        reduction = test_metrics['improvement']['api_reduction_pct']
        print(f"   {rate:.0%} Fusion Rate → {reduction:.1f}% API Reduction")

# Example sensitivity analysis
compare_fusion_rates([0.3, 0.5, 0.65, 0.8, 1.0])

## Customizable Parameters

You can modify the parameters below and re-run the analysis with different scenarios.

In [ ]:
import matplotlib.pyplot as plt

# Create comparison charts
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))

# 1. Decision Distribution
methods = ['Baseline', 'Proposed']
fusion_rates = [metrics['baseline']['fusion_rate'], metrics['proposed']['fusion_rate']]
fission_rates = [metrics['baseline']['fission_rate'], metrics['proposed']['fission_rate']]

x = range(len(methods))
width = 0.35

ax1.bar([i - width/2 for i in x], fusion_rates, width, label='Fusion', color='lightgreen')
ax1.bar([i + width/2 for i in x], fission_rates, width, label='Fission', color='lightcoral')
ax1.set_xlabel('Method')
ax1.set_ylabel('Rate')
ax1.set_title('Decision Distribution')
ax1.set_xticks(x)
ax1.set_xticklabels(methods)
ax1.legend()

# 2. Error Rates
error_rates = [metrics['baseline']['error_rate'], metrics['proposed']['error_rate']]
ax2.bar(methods, error_rates, color=['lightblue', 'orange'])
ax2.set_ylabel('Error Rate')
ax2.set_title('Error Rate Comparison')

# 3. API Calls per Example
api_calls = [metrics['baseline']['avg_calls_per_example'], metrics['proposed']['avg_calls_per_example']]
ax3.bar(methods, api_calls, color=['lightblue', 'orange'])
ax3.set_ylabel('Avg API Calls per Example')
ax3.set_title('API Efficiency')

# 4. Total API Calls
total_calls = [metrics['baseline']['api_calls'], metrics['proposed']['api_calls']]
ax4.bar(methods, total_calls, color=['lightblue', 'orange'])
ax4.set_ylabel('Total API Calls')
ax4.set_title('Total API Usage')

plt.tight_layout()
plt.show()

# Summary table
print("\n📋 Summary Comparison:")
print("-" * 60)
print(f"{'Metric':<25} {'Baseline':<12} {'Proposed':<12} {'Change':<12}")
print("-" * 60)
print(f"{'Fusion Rate':<25} {metrics['baseline']['fusion_rate']:<12.2%} {metrics['proposed']['fusion_rate']:<12.2%} {'+' if metrics['proposed']['fusion_rate'] > metrics['baseline']['fusion_rate'] else ''}{metrics['proposed']['fusion_rate'] - metrics['baseline']['fusion_rate']:<11.2%}")
print(f"{'Error Rate':<25} {metrics['baseline']['error_rate']:<12.2%} {metrics['proposed']['error_rate']:<12.2%} {'+' if metrics['improvement']['error_rate_diff'] > 0 else ''}{metrics['improvement']['error_rate_diff']:<11.2%}")
print(f"{'Avg API Calls':<25} {metrics['baseline']['avg_calls_per_example']:<12.2f} {metrics['proposed']['avg_calls_per_example']:<12.2f} {metrics['improvement']['api_reduction_pct']:<11.1f}%")

## Interactive Analysis

Let's add some interactive analysis and visualization to better understand the results.

In [ ]:
# Compute evaluation metrics
metrics = compute_metrics(results)

# Save results to JSON (optional, for compatibility with original script)
# In the notebook context, we'll just display the results instead
eval_output = json.dumps(metrics, indent=2)
print("Evaluation Results:")
print("=" * 50)
print(eval_output)

# Display key improvement metric (matching original script output)
print("\n" + "=" * 50)
print(f"🚀 API reduction: {metrics['improvement']['api_reduction_pct']:.1f}%")
print(f"📊 Error rate change: {metrics['improvement']['error_rate_diff']:.2f}")

## Run Evaluation

Now let's compute the metrics and display the results. This cell replicates the main execution logic from the original script.

In [ ]:
def compute_metrics(results: dict) -> dict:
    """Compute evaluation metrics for both baseline and proposed methods."""
    metrics = {}

    for method in ["baseline", "proposed"]:
        preds = results[method]

        # Count decisions
        fusion_count = sum(1 for p in preds if p["decision"] == "fusion")
        fission_count = sum(1 for p in preds if p["decision"] == "fission")

        # Compute error rate
        errors = sum(1 for p in preds if p["error"])
        error_rate = errors / len(preds)

        # API calls (fusion=1, fission=2)
        api_calls = fusion_count + 2 * fission_count

        metrics[method] = {
            "fusion_rate": fusion_count / len(preds),
            "fission_rate": fission_count / len(preds),
            "error_rate": error_rate,
            "api_calls": api_calls,
            "avg_calls_per_example": api_calls / len(preds),
        }

    # Compute improvement metrics
    baseline_calls = metrics["baseline"]["avg_calls_per_example"]
    proposed_calls = metrics["proposed"]["avg_calls_per_example"]
    metrics["improvement"] = {
        "api_reduction_pct": (baseline_calls - proposed_calls) / baseline_calls * 100,
        "error_rate_diff": metrics["proposed"]["error_rate"] - metrics["baseline"]["error_rate"],
    }

    return metrics

## Evaluation Metrics Function

The `compute_metrics` function analyzes the decision patterns and computes key performance indicators:

- **Fusion/Fission rates**: Percentage of each decision type
- **Error rate**: Fraction of predictions with errors
- **API efficiency**: Total API calls (fusion=1, fission=2) and average per example
- **Improvement metrics**: Comparison between baseline and proposed methods

In [ ]:
import json
import numpy as np

# Simulated evaluation results - inlined data to make notebook self-contained
# This replaces reading from "../experiment_001/method_out.json"

# Create baseline method data: all fission decisions, 8% error rate
baseline_data = []
for i in range(200):
    error_flag = i < 16  # First 16 examples have errors (8% error rate)
    baseline_data.append({
        "decision": "fission", 
        "error": error_flag
    })

# Create proposed method data: 65% fusion, 35% fission, 9% error rate  
proposed_data = []
for i in range(200):
    if i < 130:  # First 130 are fusion decisions (65%)
        decision = "fusion"
    else:  # Remaining 70 are fission decisions (35%)
        decision = "fission"
    
    error_flag = i < 18  # First 18 examples have errors (9% error rate)
    proposed_data.append({
        "decision": decision,
        "error": error_flag
    })

# Combine into the expected format
results = {
    "baseline": baseline_data,
    "proposed": proposed_data
}

print(f"Data loaded:")
print(f"- Baseline: {len(results['baseline'])} examples")
print(f"- Proposed: {len(results['proposed'])} examples")
print(f"- Sample baseline: {results['baseline'][0]}")
print(f"- Sample proposed: {results['proposed'][0]}")

## Data Setup

First, let's define our evaluation data. This simulates the results from both the baseline and proposed methods, with each prediction containing a decision type and error flag.

# DKW Controller Evaluation

This notebook evaluates the performance of a DKW (Decision, Knowledge, Workflow) Controller, comparing baseline and proposed methods. The evaluation focuses on:

- **Fusion vs Fission decisions**: How often each method chooses fusion (1 API call) vs fission (2 API calls)
- **Error rates**: Accuracy of the decision-making process  
- **API efficiency**: Total API calls and reduction percentage

This is a self-contained notebook with all data inlined for easy execution.